In [1]:
!pip install wfdb -q
!pip install scaleogram -q

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
import wfdb
from wfdb import processing
import os
import matplotlib.pyplot as plt
import wfdb_custom
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import io
import scaleogram as scg

In [2]:
### Set Dataset Path
dataset_paths = [ "/home/ec2-user/SageMaker/physionet.org/files/challenge-2016/1.0.0/training-b/" , "/home/ec2-user/SageMaker/datasets/Physionet2016_a/DenoisedEMD2016/","/home/ec2-user/SageMaker/datasets/Physionet2016_a/DenoisedEMD_ECG2016/"]
files        = [ os.listdir(dataset_paths[0]), os.listdir(dataset_paths[1]), os.listdir(dataset_paths[2])]

In [3]:
#extract .wav file names only (folder has .hea , .dat as well)
wav_files  = sorted([file for file in files[0] if '.wav' in file])
pcgmat_files  = sorted([file for file in files[1] if '.mat' in file])
ecgmat_files  = sorted([file for file in files[2] if '.mat' in file])

In [4]:
def pcgmat2np(x):
    data = scipy.io.loadmat(dataset_paths[1] + pcgmat_files[x])  # eg: 'D_A00001.mat'
    wave = data['y'] # numpy array
    wave = wave.reshape(wave.shape[0])
    return wave

In [5]:
def ecgmat2np(x):
    data = scipy.io.loadmat(dataset_paths[2] + ecgmat_files[x])  # eg: 'D_A00001.mat'
    wave = data['y'] # numpy array
    wave = wave.reshape(wave.shape[0])
    return wave

In [6]:
def plot_wavelet(a,ecg=False,save_path=None):
    
    
    scales = 0
    if(ecg):
        scales = np.arange(20, 500)
        yscaletype = 'log'
        scg.set_default_wavelet('cmor1.5-1.0')
    else:
        scales = np.arange(7, 130)
        yscaletype = 'log'
        scg.set_default_wavelet('morl')

    fs = 2000
    xcg = a
    t = np.array(range(len(xcg)-1))/fs
    cwt = scg.CWT(t, xcg,scales)
    fig = scg.cws(cwt, figsize=(9,5),yaxis='frequency',yscale = yscaletype,cscale='linear',xlabel="Time (s)",coi=False)
    
    if(save_path!=None):
        plt.savefig(save_path)
    #plt.show()
    plt.clf()
    

In [7]:
export_paths = ["/home/ec2-user/SageMaker/datasets/Physionet2016_abcdef/new_split_basedon_rawpcg/",
                "/home/ec2-user/SageMaker/datasets/Physionet2016_abcdef/new_split_basedon_rawpcg/denoised/"]

In [16]:
def metadata(x,s=100,e=14000):
    '''
        Accepts: a file id, x
        Returns: Prints parameters about a specific record
                 Plots the locally detected peaks
    
    '''
    
    rec = wav_files[x][:-4]
    
    #rec = pcgmat_files[x][2:7]
    
    record = wfdb.rdrecord(dataset_paths[0] +rec) 
    #print(dataset_paths[0])
    #print(record)
    #print(dataset_paths[0])

    #print(record.sig_name)
    #print(record.comments)
    #print(record.record_name)
    #print(record.fs)
    #print(record)
    

    a = record.p_signal
    a = a[np.logical_not(np.isnan(a))]
    a = a - np.mean(a);
    a = a.reshape(a.shape[0])
    
    print(record.p_signal.shape[0],end=" - ")
    print(a.shape[0])
    
    #b = record.p_signal[:,1] - np.mean(record.p_signal[:,1]);
    
    #return a,b
    pcg_p = wfdb.processing.find_local_peaks(a,1000) 
    #ecg_p = wfdb.processing.find_local_peaks(record.p_signal[:,1],1000) 
    #ecg_p = []
    #peaks = [pcg_p , ecg_p]
    #save_path = export_paths[0] + "pcg/lop.png"
    #print(a.shape)
    #wfdb.plot_wfdb(record=record, title='Record ' + rec + ' from PhysioNet Challenge 2016' + ' => ' + record.__dict__['comments'][0],figsize=(12,5), time_units = 'samples',ecg_grids=[1]) 
    
    #plot_wavelet(a[s:e])
    #plot_wavelet(b[s:e],ecg=True)

    
    '''
    ##Denoised
    pcg_wave = pcgmat2np(x)
    ecg_wave = ecgmat2np(x)
    pcg_wave = pcg_wave - np.mean(pcg_wave)
    ecg_wave = ecg_wave - np.mean(ecg_wave)
    
    '''
    if(len(a)==0):
        print("==== ERROR ==== ", x)
        return
    
    
    #pcg_p = wfdb.processing.find_local_peaks(pcg_wave,1000) 
    #ecg_p = wfdb.processing.find_local_peaks(ecg_wave,1000) 
    #peaks = [pcg_p , ecg_p] 
    
    #wave = [pcg_wave , ecg_wave]
    
    
    #print("Denoised Waves")
    #wfdb_custom.custom_plot_signal(s,e,peaks,wave,figsize=(12,5))
    #plot_wavelet(pcg_wave[s:e] )
    #plot_wavelet(ecg_wave[s:e],ecg=True)
    
    
    e_p = -1
    
    
    #Scalogram Generation
    for i in range(0,len(pcg_p)):

        s_p = pcg_p[i]
        if(s_p < e_p):
            continue
        
        e_p   = s_p + 7000
        #print(s_p,e_p)
        
        rec = wav_files[x][:-4]
        if(record.comments[0]=='Abnormal'):
            rec = rec + "_" + str(s_p) + '_A.tiff'
        else:
            rec = rec + "_" + str(s_p) + '_N.tiff'
         
        plot_wavelet(a[s_p:e_p]   ,save_path=export_paths[0] + 'pcg/' + rec )
        #plot_wavelet(b[s_p:e_p]   ,ecg=True,save_path=export_paths[0] + 'ecg/' + rec )
        
        #plot_wavelet(pcg_wave[s_p:e_p],save_path=export_paths[1] + 'pcg/' + rec )
        #plot_wavelet(ecg_wave[s_p:e_p],ecg=True,save_path=export_paths[1] + 'ecg/' + rec )
        
   
    
     

In [24]:
names = ['b','c','d','e','f']

for k in range(0,1):
    dataset_paths[0] = "/home/ec2-user/SageMaker/physionet.org/files/challenge-2016/1.0.0/training-" + names[k] + "/"
    files        = [ os.listdir(dataset_paths[0]), os.listdir(dataset_paths[1]), os.listdir(dataset_paths[2])]
    wav_files    = sorted([file for file in files[0] if '.wav' in file])
    #print(dataset_paths[0])

    for i in range(485,len(wav_files)):
        print(wav_files[i],end=" -_-")
        metadata(i)
        #summ  = summ + metadata(i)


b0486.wav -_-16000 - 15999
b0487.wav -_-16000 - 15999
b0488.wav -_-16000 - 16000
b0489.wav -_-16000 - 15999
b0490.wav -_-16000 - 15999


<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

...

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

<Figure size 648x360 with 0 Axes>

In [ ]:
for i in range(41,409):
    metadata(i)
    print(i)

41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [ ]:
#PCG => Morlet 

#ECG => Cmor1.5-1.0

'''
    if(ecg):
        scales = np.arange(2, 450)
        yscaletype = 'log'
        scg.set_default_wavelet('cmor1.5-1.0')
    else:
        scales = np.arange(7, 130)
        yscaletype = 'log'
        scg.set_default_wavelet('morl')
'''



In [ ]:
wave = mat2np(26)
plt.plot(wave)

In [14]:
count = 0 
for x in range(0,len(pcgmat_files)):
    pcg_wave = pcgmat2np(x)
    ecg_wave = ecgmat2np(x)
    pcg_wave = pcg_wave - np.mean(pcg_wave)
    ecg_wave = ecg_wave - np.mean(ecg_wave)
    
    if(len(pcg_wave)==0 or len(ecg_wave)==0):
        count = count+1
        print("==== ERROR ==== ", x)
    

==== ERROR ====  12
==== ERROR ====  13
==== ERROR ====  17
==== ERROR ====  26
==== ERROR ====  27
==== ERROR ====  43
==== ERROR ====  53
==== ERROR ====  55
==== ERROR ====  66
==== ERROR ====  68
==== ERROR ====  73
==== ERROR ====  88
==== ERROR ====  111
==== ERROR ====  115
==== ERROR ====  134
==== ERROR ====  156
==== ERROR ====  157
==== ERROR ====  160
==== ERROR ====  176
==== ERROR ====  182
==== ERROR ====  201
==== ERROR ====  224
==== ERROR ====  245
==== ERROR ====  256
==== ERROR ====  268
==== ERROR ====  269
==== ERROR ====  271
==== ERROR ====  274
==== ERROR ====  286
==== ERROR ====  290
==== ERROR ====  298
==== ERROR ====  300
==== ERROR ====  306
==== ERROR ====  309
==== ERROR ====  310
==== ERROR ====  315
==== ERROR ====  328
==== ERROR ====  332
==== ERROR ====  339
==== ERROR ====  342
==== ERROR ====  356
==== ERROR ====  361
==== ERROR ====  362
==== ERROR ====  390
==== ERROR ====  395


In [15]:
count

45

In [ ]:
#!aws s3 cp /home/ec2-user/SageMaker/datasets s3://intelliscope-data/datasets --recursive